<a href="https://colab.research.google.com/github/narfdf999/Arquitectura/blob/master/PMV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Primera aproximación al problema de generar edificio a lo largo de un camino dibujado por el arquitecto

Importar librerías

In [0]:
import tensorflow as tf
import numpy as np
import pandas as pd
import csv
import matplotlib.pyplot as plt
%matplotlib inline
import keras
from keras.layers import Dense, Dropout, Input
from keras.models import Model,Sequential
from tqdm import tqdm
from keras.layers.advanced_activations import LeakyReLU
from keras.optimizers import Adam

Visualización de datos

In [0]:
a=np.genfromtxt('17_a.csv',delimiter=',')

In [44]:
b=pd.read_csv('17_a.csv',sep=',')
b


,-0.013502617649277737,0.9216023703084879,-0.5857466785447467,"0.996373453486285,0.0850878439485549,0",0,-0.923764078523375,-0.916547031969692,-0.903993777080012,-0.884629654218985,-0.856170444070877,-0.815172307666003,-0.755939116856985,-0.666458497007676,-0.518636628673306,-0.522421885079881,1,-0.252698655933787,-0.695299716717548,-0.822562746130715,-0.880466295791138,-0.911661435231547,-0.929698527207505,-0.940190704955078,-0.945784977372277,-0.94776374744804,-0.926459898520273,-0.923764078523375.1,-0.916547031969692.1,-0.903993777080012.1,-0.884629654218985.1,-0.856170444070877.1,-0.815172307666003.1,-0.806365327636463,-0.81070189403321,-0.813231410466662,-0.814700086818437,-0.815339560869832,-0.814903398465021,-0.812583842230374,-0.822562746130715.1,...,-0.916547031969692.57,-0.903993777080012.46,-0.884629654218984.11,-0.856170444070877.28,-0.847313882008489,-0.853086206753915,-0.856376430613274,-0.858295616107177,-0.859409924878722,-0.85989510430153,-0.859564180750598,-0.857804295493892,-0.853418051155239,-0.880466295791138.31,-0.911661435231547.42,-0.929698527207505.55,-0.940190704955078.58,-0.945784977372277.58,-0.94776374744804.58,-0.926459898520273.58,-0.923764078523375.59,-0.916547031969692.58,-0.903993777080012.47,-0.884629654218985.23,-0.856170444070877.29,-0.815172307666003.7,-0.757891587752352,-0.76331375307072,-0.766476498667787,-0.768312837662729,-0.769112395125073,-0.768567045544742,-0.76566682075424,-0.822562746130715.9,-0.880466295791138.32,-0.911661435231547.43,-0.929698527207505.56,-0.940190704955078.59,-0.945784977372277.59,-0.94776374744804.59
0,-0.011626,0.921837,-0.578242,"0.996503813904489,0.0835472852569602,8.3474960...",0.000981,-0.923764,-0.916547,-0.903994,-0.884630,-0.856170,-0.815172,-0.755939,-0.666458,-0.515757,-0.519565,1.000000,-0.252699,-0.695300,-0.822563,-0.880466,-0.911661,-0.929699,-0.940191,-0.945785,-0.947764,-0.926460,-0.923764,-0.916547,-0.903994,-0.884630,-0.856170,-0.815172,-0.803948,-0.808339,-0.810900,-0.812387,-0.813035,-0.812593,-0.810244,-0.822563,...,-0.916547,-0.903994,-0.884630,-0.856170,-0.844142,-0.850034,-0.853393,-0.855352,-0.856489,-0.856984,-0.856646,-0.854850,-0.850373,-0.880466,-0.911661,-0.929699,-0.940191,-0.945785,-0.947764,-0.926460,-0.923764,-0.916547,-0.903994,-0.884630,-0.856170,-0.815172,-0.755939,-0.760379,-0.763581,-0.765440,-0.766249,-0.765697,-0.762761,-0.822563,-0.880466,-0.911661,-0.929699,-0.940191,-0.945785,-0.947764
1,-0.009748,0.922071,-0.570736,"0.996635673957332,0.0819593398924993,-3.833137...",0.001963,-0.923764,-0.916547,-0.903994,-0.884630,-0.856170,-0.815172,-0.755939,-0.666458,-0.512877,-0.516707,1.000000,-0.252699,-0.695300,-0.822563,-0.880466,-0.911661,-0.929699,-0.940191,-0.945785,-0.947764,-0.926460,-0.923764,-0.916547,-0.903994,-0.884630,-0.856170,-0.815172,-0.801576,-0.806020,-0.808612,-0.810117,-0.810772,-0.810325,-0.807948,-0.822563,...,-0.916547,-0.903994,-0.884630,-0.856170,-0.840969,-0.846981,-0.850408,-0.852407,-0.853568,-0.854073,-0.853728,-0.851895,-0.847327,-0.880466,-0.911661,-0.929699,-0.940191,-0.945785,-0.947764,-0.926460,-0.923764,-0.916547,-0.903994,-0.884630,-0.856170,-0.815172,-0.755939,-0.757444,-0.760685,-0.762567,-0.763386,-0.762827,-0.759855,-0.822563,-0.880466,-0.911661,-0.929699,-0.940191,-0.945785,-0.947764
2,-0.007871,0.922305,-0.563229,"0.996769003100824,0.0803215690670275,-6.129462...",0.002944,-0.923764,-0.916547,-0.903994,-0.884630,-0.856170,-0.815172,-0.755939,-0.666458,-0.511259,-0.513850,1.000000,-0.252699,-0.695300,-0.822563,-0.880466,-0.911661,-0.929699,-0.940191,-0.945785,-0.947764,-0.926460,-0.923764,-0.916547,-0.903994,-0.884630,-0.856170,-0.815172,-0.799250,-0.803746,-0.806369,-0.807891,-0.808554,-0.808102,-0.805697,-0.822563,...,-0.916547,-0.903994,-0.884630,-0.856170,-0.837796,-0.843928,-0.847423,-0.849462,-0.850646,-0.851161,-0.850810,-0.848940,-0.844281,-0.880466,-0.911661,-0.929699,-0.940191,-0.945785,-0.947764,-0.926460,-0.923764,-0.916547,-0.903994,-0.884630,-0.856170,-0.815172,-0.755939,-

In [0]:
x=a[:,0]
#print(x)
y=a[:,1]
#print(y)
z=a[:,2]
#print(z)
t=a[:,6]
#print(t)

p=np.vstack((x,y,z,t)).transpose()


d=a[:,7:1266]




Preparación y carga de datos

Optimizador ADAM


In [0]:
def adam_optimizer():
    return Adam(lr=0.0002, beta_1=0.5)

Generador

In [47]:
def create_generator():
  generator=Sequential()
  generator.add(Dense(units=256,input_dim=132))
  generator.add(LeakyReLU(0.2))

  generator.add(Dense(units=512))
  generator.add(LeakyReLU(0.2))

  generator.add(Dense(units=1024))
  generator.add(LeakyReLU(0.2))

  generator.add(Dense(units=1259, activation='tanh'))
  #poner la salida a 1259 

  generator.compile(loss='binary_crossentropy', optimizer=adam_optimizer())
  return generator
  
g=create_generator()
g.summary()
  



Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_23 (Dense)             (None, 256)               34048     
_________________________________________________________________
leaky_re_lu_19 (LeakyReLU)   (None, 256)               0         
_________________________________________________________________
dense_24 (Dense)             (None, 512)               131584    
_________________________________________________________________
leaky_re_lu_20 (LeakyReLU)   (None, 512)               0         
_________________________________________________________________
dense_25 (Dense)             (None, 1024)              525312    
_________________________________________________________________
leaky_re_lu_21 (LeakyReLU)   (None, 1024)              0         
_________________________________________________________________
dense_26 (Dense)             (None, 1259)             

Discriminador

In [48]:
def create_discriminator():
    discriminator=Sequential()
    discriminator.add(Dense(units=1024,input_dim=1263))
    discriminator.add(LeakyReLU(0.2))
       
    discriminator.add(Dense(units=512))
    discriminator.add(LeakyReLU(0.2))
       
    discriminator.add(Dense(units=256))
    discriminator.add(LeakyReLU(0.2))

    discriminator.add(Dense(units=128))
    discriminator.add(LeakyReLU(0.2))

    discriminator.add(Dense(units=64))
    discriminator.add(LeakyReLU(0.2))

    discriminator.add(Dense(units=32))
    discriminator.add(LeakyReLU(0.2))
    
    discriminator.add(Dense(units=1, activation='softmax'))
    
    discriminator.compile(loss='binary_crossentropy', optimizer=adam_optimizer())
    return discriminator
d =create_discriminator()
d.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_27 (Dense)             (None, 1024)              1294336   
_________________________________________________________________
leaky_re_lu_22 (LeakyReLU)   (None, 1024)              0         
_________________________________________________________________
dense_28 (Dense)             (None, 512)               524800    
_________________________________________________________________
leaky_re_lu_23 (LeakyReLU)   (None, 512)               0         
_________________________________________________________________
dense_29 (Dense)             (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_24 (LeakyReLU)   (None, 256)               0         
_________________________________________________________________
dense_30 (Dense)             (None, 128)              

GAN

In [52]:
def create_gan(discriminator, generator):
  discriminator.trainable=False
  gan_input = Input(shape=(132,))
  x = generator(gan_input)
  p = Input(shape=(4,))
  x = keras.layers.Concatenate()([x,p])
  gan_output=discriminator(x)
  gan = Model(inputs=[gan_input, p], output=gan_output)
  gan.compile(loss='binary_crossentropy', optimizer='adam')
  return gan

gan=create_gan(d,g)
gan.summary()

Model: "model_8"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_37 (InputLayer)           (None, 132)          0                                            
__________________________________________________________________________________________________
sequential_5 (Sequential)       (None, 1259)         1981419     input_37[0][0]                   
__________________________________________________________________________________________________
input_38 (InputLayer)           (None, 4)            0                                            
__________________________________________________________________________________________________
concatenate_23 (Concatenate)    (None, 1263)         0           sequential_5[4][0]               
                                                                 input_38[0][0]             

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("se...)`
  


In [0]:
X_train = np.random.random((128,1263))
Y_train = np.random.random((128,1))
X_test = np.random.random((128,1263))
Y_test = np.random.random((128,1))
print(X_train)
print(Y_train)
print(X_test)
print(Y_test)

[[0.06312674 0.72860588 0.2082473  ... 0.640445   0.35728622 0.56532665]
 [0.09503372 0.24187864 0.08025132 ... 0.00359826 0.59389998 0.85535053]
 [0.10693559 0.85417473 0.9155465  ... 0.25495492 0.65015428 0.99502202]
 ...
 [0.38454622 0.52872142 0.23196792 ... 0.03379047 0.77315599 0.97443506]
 [0.34912307 0.29348685 0.36991584 ... 0.48052202 0.38361784 0.13326973]
 [0.3653524  0.5983976  0.41273885 ... 0.75254954 0.37453193 0.40053536]]
[[0.3891861 ]
 [0.18290104]
 [0.13658893]
 [0.15507847]
 [0.33709142]
 [0.9911378 ]
 [0.84304467]
 [0.67673892]
 [0.24671599]
 [0.35464414]
 [0.6818177 ]
 [0.18815264]
 [0.25045665]
 [0.47689544]
 [0.85937737]
 [0.01367633]
 [0.9071803 ]
 [0.52774284]
 [0.30220463]
 [0.04629105]
 [0.87441471]
 [0.41215694]
 [0.98191275]
 [0.66807323]
 [0.87781774]
 [0.21400492]
 [0.14920834]
 [0.51302766]
 [0.82493305]
 [0.38886303]
 [0.61994116]
 [0.85176949]
 [0.99841436]
 [0.42744744]
 [0.05029562]
 [0.48522952]
 [0.97048745]
 [0.47113416]
 [0.74480124]
 [0.799827

Entrenamiento

In [0]:
def training(epochs=1, batch_size=12):
    
    #Loading the data
    #(X_train, y_train, X_test, y_test) = load_data()
    batch_count = X_train.shape[0] / batch_size
    
    # Creating GAN
    generator= create_generator()
    discriminator= create_discriminator()
    gan = create_gan(discriminator, generator)
    
    for e in range(1,epochs+1 ):
        print("Epoch %d" %e)
        for _ in tqdm(range(batch_size)):
        #generate  random noise as an input  to  initialize the  generator
            noise= np.random.normal(0,1, [batch_size, 132])
            # cambiar a 128
            
            # Generate fake MNIST images from noised input
            generated_images = generator.predict(noise)
            print(generated_images.shape)
            
            # Get a random set of  real images
            image_batch =X_train[np.random.randint(low=0,high=X_train.shape[0],size=batch_size)]
            print(image_batch.shape)
            
            #Construct different batches of  real and fake data 
            X= np.concatenate([image_batch, generated_images])
            
            # Labels for generated and real data
            y_dis=np.zeros(2*batch_size)
            y_dis[:batch_size]=0.9
            
            #Pre train discriminator on  fake and real data  before starting the gan. 
            discriminator.trainable=True
            discriminator.train_on_batch(X, y_dis)
            
            #Tricking the noised input of the Generator as real data
            noise= np.random.normal(0,1, [batch_size, 132])
            y_gen = np.ones(batch_size)
            
            # During the training of gan, 
            # the weights of discriminator should be fixed. 
            #We can enforce that by setting the trainable flag
            discriminator.trainable=False
            
            #training  the GAN by alternating the training of the Discriminator 
            #and training the chained GAN model with Discriminator’s weights freezed.
            gan.train_on_batch(noise, y_gen)
            
        if e == 1 or e % 20 == 0:
           
            plot_generated_images(e, generator)
training(400,128)

NameError: ignored

In [0]:
generator

NameError: ignored

Postprocesado de datos de salida

In [0]:
df = pd.read_csv('17_a.csv', header=None)

In [0]:
df[0]=df[0].apply(lambda x: x.split(','))

In [0]:
df[0]

0       [13.3780471856351, 2.8377602455671, 1.48631674...
1       [13.4278691159807, 2.84197631680024, 1.4863167...
2       [13.477697602257, 2.8461141852625, 1.486316745...
3       [13.5275327187439, 2.8501714213576, 1.48631674...
4       [13.5773745379712, 2.85414546918325, 1.4863167...
                              ...                        
1014    [18.3546830379293, 1.41693950216082, 3.6487514...
1015    [18.3878800275203, 1.37955020045437, 3.6487514...
1016    [18.4207616530758, 1.34188325604917, 3.6487514...
1017    [18.4533341781645, 1.30394869912087, 3.6487514...
1018    [18.4856037816336, 1.26575612534079, 3.6487514...
Name: 0, Length: 1019, dtype: object